# Dev 07 - Wind-Turbine Location

In [ ]:
import csv
import time
import numpy as np
import scipy.signal
import matplotlib
import matplotlib.pyplot as plt

import cspec
import toshi

In [ ]:
# plt.style.use('./darkmode.style')
zmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.zmap()[:, :3])
vmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.vmap()[:, :3])

In [ ]:
def wtloc_from_csv(filename):
    data = []
    with open(filename) as file:
        reader = csv.reader(file, delimiter=',')
        for k, row in enumerate(reader):
            data.append(row)
    return np.array(data, dtype=np.single)

In [ ]:
loc = wtloc_from_csv('wtloc.csv')

In [ ]:
loc

In [ ]:
file = os.path.expanduser('~/Downloads/20200614_150015.256007-87-02.iqData.XXXX.AKITA.dat')
filesize = os.path.getsize(file)

In [ ]:
s = time.time()
ray_pulses, cpi_headers = toshi.read(file)
e = time.time()
print('Data read in {:.2f} s'.format(e - s))

# Go through the pulses for azimuth
a = np.zeros(len(ray_pulses), dtype=np.single)
for k, pulses in enumerate(ray_pulses):
    a[k] = pulses[0].azimuth
    
# Choose ray 2 to whatever that completes the 360-deg coverage
n = np.argmin(np.abs(a[3:] - a[2])) + 1
a = a[2:n+2]
ray_pulses = ray_pulses[2:n+2]
cpi_headers = cpi_headers[2:n+2]

In [ ]:
# Dimensions
naz = len(ray_pulses)
ngate_long = ray_pulses[0][0].ngate_long_hi
ngate_short = ray_pulses[0][0].ngate_short_hi
ngate = ngate_long + ngate_short

# Elevation assumed to be flat from the very first pulse
scan_el = ray_pulses[0][0].elevation
scan_time = time.strptime(os.path.basename(file)[:15], '%Y%m%d_%H%M%S')

# Sampling code from the CPI header
fs = 1.0e6 * (1 << cpi_headers[0].fs_code)
dr = 3.0e8 / fs / 2
r = 1.0e-3 * np.arange(0, ngate, dtype=np.single) * dr

In [ ]:
# Noise estimate, try azimuth 0, around 20-25 km
ia = np.argmin(np.abs(a[3:] - 0.0)) + 1
ir, er = np.argmin(np.abs(r - 20.0)), np.argmin(np.abs(r - 25.0))
samples = np.zeros((len(ray_pulses[ia]), er-ir), dtype=np.csingle)

# Gather the samples. Ignore phase code since we are only interested in amplitude
for k, pulse in enumerate(ray_pulses[ia]):
    samples[k, :] = pulse.h_long_hi[ir:er]
# noise = np.mean(np.abs(samples)) ** 2
noise = 24
print('Noise estimate in 16-bit ADU: {:.4f}'.format(noise))

In [ ]:
s = np.zeros((naz, ngate), dtype=np.single)
v = np.zeros((naz, ngate), dtype=np.single)

nfft = 256
use_window = True
use_pulse_pair = False

# Go through the pulses
a = np.zeros(len(ray_pulses), dtype=np.single)
for k, (pulses, cpi_header) in enumerate(zip(ray_pulses, cpi_headers)):
    p = np.zeros((len(pulses), ngate), dtype=np.csingle)
    a[k] = pulses[0].azimuth
    for j, pulse in enumerate(pulses):
        p[j, ngate_short:] = pulse.h_long_hi * np.exp(-1j * pulse.phase_h_long)
        p[j, :ngate_short] = pulse.h_short_hi * np.exp(-1j * pulse.phase_h_short)
    p = p - np.mean(p, axis=0)                                  # Remove DC if desired

    if use_window:
        w = scipy.signal.get_window('blackmanharris', len(p)).reshape((p.shape[0],))
    else:
        w = np.ones((p.shape[0],))
    w /= np.sqrt(np.sum(w ** 2)) / np.sqrt(p.shape[0])         # Normalize to non-windowed gain
    ww = np.repeat(np.expand_dims(w, axis=1), ngate, axis=1)   # Make same shape
    p *= ww                                                    # Windowing

    if use_pulse_pair:
        pp = p[1:, :] * np.conj(p[:-1, :])                      # x(n) * x'(n-1)
        s[k, :] = np.mean(np.abs(p) ** 2, axis=0)               # s(n) = E[x(n) * x'(n)]
        v[k, :] = np.angle(np.sum(pp, axis=0))                  # r(1) = E[x(n) * x'(n-1)]
    else:
        spec = np.fft.fft(p, nfft, axis=0)                                 # FFT
        s[k, :] = np.mean(np.abs(spec) ** 2, axis=0) / p.shape[0]          # Periodogram
        v[k, :] = np.angle(np.fft.ifft(spec * np.conj(spec), axis=0)[1])   # IFT -> ACF[1]

# Signal
s -= noise
s[s <= 0] = 1.0e-6                                    # Avoid log(0)
snr = 10 * np.log10(s / noise)                        # Signal-to-noise ratio in dB
z = 10 * np.log10(s * (r + 0.5e-3 * dr) ** 2) - 40    # Estimated ZCal = -40 
z[:, :ngate_short] += 18                              # ~18-dB on the short waveform, perhaps?

# Thresholding at SNR = 0 dB
m = snr < 0
z[m] = np.nan
v[m] = np.nan

In [ ]:
turbines = np.array([[pt[2], pt[1]] for pt in loc], dtype=np.single)

In [ ]:
# Edge of range cells
wid_a = np.mean(sorted(np.diff(a))[int(0.3 * len(a)):int(0.6 * len(a))])
end_a = a[-1] + wid_a
if end_a >= 360.0:
    end_a -= 360.0
ae = np.append(a, end_a)
re = 1.0e-3 * np.arange(0, ngate + 1, dtype=np.single) * dr

In [ ]:
# toshi.turbines = importlib.reload(toshi.turbines)
# toshi.turbine_cells_2 = toshi.turbines.turbine_cells_2

# Radar cell locations
ce = np.cos(np.deg2rad(scan_el))
rr, aa = np.meshgrid(re, np.deg2rad(ae))
xx = rr * ce * np.sin(aa)
yy = rr * ce * np.cos(aa)

# Turbine locations
# turbines = toshi.turbine_cells_2
a_turb = turbines[:, 0] / 180.0 * np.pi
r_turb = turbines[:, 1]
x_turb = r_turb * np.sin(a_turb)
y_turb = r_turb * np.cos(a_turb)

# Various domain to choose from
# xlim, ylim = (-3, 2), (2, 6.5)
# xlim, ylim = (-7, 6.5), (-6, 7)
# xlim, ylim = (-13, 5), (3, 18)
# xlim, ylim = (-25, 10), (0, 35)
xlim, ylim = (-20, 20), (-20, 20)

# Reflectivity plot
plt.figure(figsize=(5.2, 4), dpi=200)
plt.pcolormesh(xx[:, :501], yy[:, :501], z[:, :500], cmap=zmap)
plt.plot(x_turb, y_turb, 'xk', markersize=4)
plt.clim((-32, 96))
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title(os.path.basename(file), fontsize=8)

# Velocity plot
plt.figure(figsize=(5.2, 4), dpi=200)
plt.pcolormesh(xx[::2, :501], yy[::2, :501], v[::2, :500], cmap=vmap)
plt.plot(x_turb, y_turb, 'xk', markersize=4)
plt.clim((-5, 5))
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title(os.path.basename(file), fontsize=8)

plt.show()

In [ ]:
turbines.shape